In [1]:
# GEE Analysis with FastAPI Integration and MapStore Catalog
# This notebook demonstrates:
# 1. GEE Authentication using GCP service account
# 2. Creating cloudless Sentinel-2 imagery composite
# 3. Pushing results to FastAPI service
# 4. Making layers available for MapStore catalog

import ee
import json
import os
import sys
import requests
from datetime import datetime
import folium

# Add gee_lib to path
sys.path.insert(0, '/usr/src/app/gee_lib')

print("✓ Imports loaded successfully")

✓ Imports loaded successfully


In [2]:
# Step 1: Initialize Google Earth Engine with Service Account
def initialize_gee():
    """Initialize GEE using service account credentials"""
    try:
        # Load credentials from user_id.json
        credentials_path = '/usr/src/app/user_id.json'
        
        with open(credentials_path, 'r') as f:
            credentials_data = json.load(f)
        
        service_account = credentials_data['client_email']
        
        # Initialize Earth Engine
        credentials = ee.ServiceAccountCredentials(service_account, credentials_path)
        ee.Initialize(credentials)
        
        print(f"✓ GEE Initialized successfully")
        print(f"  Service Account: {service_account}")
        print(f"  Project ID: {credentials_data['project_id']}")
        
        return True
    except Exception as e:
        print(f"✗ Error initializing GEE: {e}")
        return False

# Initialize GEE
initialize_gee()


✓ GEE Initialized successfully
  Service Account: earth-engine-land-eligibility@ee-iwansetiawan.iam.gserviceaccount.com
  Project ID: ee-iwansetiawan


True

In [3]:
# Step 2: Define Area of Interest (AOI)
# Example: A region in Indonesia (Kalimantan)
aoi_coords = [
    [109.5, -1.5],
    [110.5, -1.5],
    [110.5, -0.5],
    [109.5, -0.5],
    [109.5, -1.5]
]

aoi = ee.Geometry.Polygon(aoi_coords)
aoi_center = aoi.centroid().coordinates().getInfo()

print("✓ AOI defined")
print(f"  Center coordinates: {aoi_center}")
print(f"  Area: {aoi.area().divide(1e6).getInfo():.2f} km²")


✓ AOI defined
  Center coordinates: [109.99999999999977, -1.000012676912049]
  Area: 12362.61 km²


In [4]:
# Step 3: Create Cloudless Sentinel-2 Composite using gee_lib
from osi.image_collection.main import ImageCollection

# Configuration
config = {
    'IsThermal': False,
}

date_start_end = ['2023-01-01', '2023-12-31']
cloud_cover_threshold = 20  # Maximum cloud cover percentage

print("Creating cloudless Sentinel-2 composite...")
print(f"  Date range: {date_start_end[0]} to {date_start_end[1]}")
print(f"  Cloud cover threshold: {cloud_cover_threshold}%")

# Create ImageCollection instance
img_collection = ImageCollection(
    I_satellite='Sentinel',
    region='asia',
    AOI=aoi,
    date_start_end=date_start_end,
    cloud_cover_threshold=cloud_cover_threshold,
    config=config
)

# Get cloud-masked image collection
sentinel_collection = img_collection.image_collection_mask()

# Create median composite (cloudless)
sentinel_composite = img_collection.image_mosaick()

print("✓ Sentinel-2 cloudless composite created")
print(f"  Number of images used: {sentinel_collection.size().getInfo()}")


Creating cloudless Sentinel-2 composite...
  Date range: 2023-01-01 to 2023-12-31
  Cloud cover threshold: 20%
selecting Sentinel images


selecting Sentinel images
✓ Sentinel-2 cloudless composite created
  Number of images used: 25


In [5]:
# Step 4: Generate Analysis Products
# Create True Color, False Color, and NDVI visualizations

# True Color RGB (Natural Color)
true_color = sentinel_composite.select(['red', 'green', 'blue'])

# False Color (NIR, Red, Green) - highlights vegetation
false_color = sentinel_composite.select(['nir', 'red', 'green'])

# Calculate NDVI
ndvi = sentinel_composite.normalizedDifference(['nir', 'red']).rename('NDVI')

# Calculate EVI (Enhanced Vegetation Index)
evi = sentinel_composite.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': sentinel_composite.select('nir'),
        'RED': sentinel_composite.select('red'),
        'BLUE': sentinel_composite.select('blue')
    }
).rename('EVI')

# Calculate NDWI (Normalized Difference Water Index)
ndwi = sentinel_composite.normalizedDifference(['green', 'nir']).rename('NDWI')

print("✓ Analysis products generated:")
print("  - True Color RGB")
print("  - False Color Composite")
print("  - NDVI (Normalized Difference Vegetation Index)")
print("  - EVI (Enhanced Vegetation Index)")
print("  - NDWI (Normalized Difference Water Index)")


✓ Analysis products generated:
  - True Color RGB
  - False Color Composite
  - NDVI (Normalized Difference Vegetation Index)
  - EVI (Enhanced Vegetation Index)
  - NDWI (Normalized Difference Water Index)


In [6]:
# Step 5: Create GEE Map IDs for Tile Serving
# These Map IDs can be used to serve tiles via URLs

# Visualization parameters
vis_params = {
    'true_color': {
        'bands': ['red', 'green', 'blue'],
        'min': 0,
        'max': 0.3,
        'gamma': 1.4
    },
    'false_color': {
        'bands': ['nir', 'red', 'green'],
        'min': 0,
        'max': 0.5,
        'gamma': 1.4
    },
    'ndvi': {
        'min': -0.2,
        'max': 0.8,
        'palette': ['red', 'yellow', 'green', 'darkgreen']
    },
    'evi': {
        'min': -0.2,
        'max': 0.8,
        'palette': ['brown', 'yellow', 'lightgreen', 'darkgreen']
    },
    'ndwi': {
        'min': -0.3,
        'max': 0.3,
        'palette': ['white', 'lightblue', 'blue', 'darkblue']
    }
}

# Generate Map IDs
print("Generating GEE Map IDs...")

map_ids = {}
map_ids['true_color'] = true_color.getMapId(vis_params['true_color'])
map_ids['false_color'] = false_color.getMapId(vis_params['false_color'])
map_ids['ndvi'] = ndvi.getMapId(vis_params['ndvi'])
map_ids['evi'] = evi.getMapId(vis_params['evi'])
map_ids['ndwi'] = ndwi.getMapId(vis_params['ndwi'])

print("✓ Map IDs generated for all layers")

# Display tile URLs
for layer_name, map_id_dict in map_ids.items():
    tile_url = map_id_dict['tile_fetcher'].url_format
    print(f"\n{layer_name.upper()}:")
    print(f"  Tile URL: {tile_url}")


Generating GEE Map IDs...
✓ Map IDs generated for all layers

TRUE_COLOR:
  Tile URL: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/6951b6c1d53c24dc168a5d32691b2608-325a0c6a0822cbed1b71fcc38ba44a8b/tiles/{z}/{x}/{y}

FALSE_COLOR:
  Tile URL: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/18403c46c469076da51c47cdfda5d68c-da9b200980e2ccd6595e1505f791434b/tiles/{z}/{x}/{y}

NDVI:
  Tile URL: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/b44fc7b2e551722b03eaa4c338dcea9d-6a26226bca238ef080f99d50528b1667/tiles/{z}/{x}/{y}

EVI:
  Tile URL: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/36c8d9f9fe7524ac37247e557cb5c1e3-52b4a309a225f5ffd4ea1ee7aa921120/tiles/{z}/{x}/{y}

NDWI:
  Tile URL: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/decdef08c51f7c0bc2a9bb1e7c630932-72723617fce3df7a79951323b372fa52/tiles/{z}/{x}/{y}


In [7]:
# Step 6: Visualize on Interactive Map using Folium
# Create an interactive map centered on AOI

# Create base map
center_lat, center_lon = aoi_center[1], aoi_center[0]
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add GEE tile layers
for layer_name, map_id_dict in map_ids.items():
    tile_url = map_id_dict['tile_fetcher'].url_format
    folium.TileLayer(
        tiles=tile_url,
        attr='Google Earth Engine',
        name=layer_name.replace('_', ' ').title(),
        overlay=True,
        control=True
    ).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

print("✓ Interactive map created with all layers")
print(f"  Map centered at: ({center_lat:.4f}, {center_lon:.4f})")

# Display the map
m


✓ Interactive map created with all layers
  Map centered at: (-1.0000, 110.0000)


In [8]:
# Step 7: Prepare Data for FastAPI Service
# Create a payload with layer information and metadata

project_id = f"sentinel_analysis_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

# Prepare layer data for FastAPI
layers_data = {
    'project_id': project_id,
    'project_name': 'Sentinel-2 Cloudless Composite Analysis',
    'aoi': {
        'type': 'Polygon',
        'coordinates': [aoi_coords],
        'center': aoi_center
    },
    'date_range': {
        'start': date_start_end[0],
        'end': date_start_end[1]
    },
    'analysis_params': {
        'satellite': 'Sentinel-2',
        'cloud_cover_threshold': cloud_cover_threshold,
        'image_count': sentinel_collection.size().getInfo()
    },
    'layers': {}
}

# Add layer information
for layer_name, map_id_dict in map_ids.items():
    layers_data['layers'][layer_name] = {
        'name': layer_name.replace('_', ' ').title(),
        'description': f'{layer_name.upper()} visualization from Sentinel-2',
        'tile_url': map_id_dict['tile_fetcher'].url_format,
        'map_id': map_id_dict['mapid'],
        'token': map_id_dict['token'],
        'vis_params': vis_params[layer_name]
    }

print("✓ Data prepared for FastAPI")
print(f"  Project ID: {project_id}")
print(f"  Total layers: {len(layers_data['layers'])}")


✓ Data prepared for FastAPI
  Project ID: sentinel_analysis_20251020_174835
  Total layers: 5


In [9]:
# Step 8: Push Results to FastAPI Service
# Send the analysis results to FastAPI so they can be served to MapStore

# FastAPI service URL (within Docker network)
fastapi_url = "http://fastapi:8000"

def push_to_fastapi(data, endpoint="/layers/register"):
    """Push GEE analysis results to FastAPI service"""
    try:
        # Send POST request to FastAPI
        url = f"{fastapi_url}{endpoint}"
        print(f"Pushing to: {url}")
        
        response = requests.post(
            url,
            json=data,
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            print("✓ Successfully pushed to FastAPI")
            print(f"  Status: {result.get('status')}")
            print(f"  Project ID: {result.get('project_id')}")
            print(f"  Layers Count: {result.get('layers_count')}")
            print(f"  Message: {result.get('message')}")
            return result
        else:
            print(f"✗ Error: {response.status_code}")
            try:
                error_detail = response.json()
                print(f"  Detail: {error_detail.get('detail', response.text)}")
            except:
                print(f"  Response: {response.text}")
            return None
    except requests.exceptions.ConnectionError as e:
        print(f"✗ Connection Error: Cannot reach FastAPI service")
        print(f"  Make sure FastAPI container is running")
        print(f"  Check: docker ps | grep fastapi")
        return None
    except Exception as e:
        print(f"✗ Error pushing to FastAPI: {e}")
        import traceback
        print(f"  Traceback: {traceback.format_exc()}")
        return None

# Push data to FastAPI (using the new register endpoint)
print("Registering layers with FastAPI...")
result = push_to_fastapi(layers_data)

if result:
    print("\n📡 Data is now available in FastAPI service")
    print(f"   Access layers: {fastapi_url}/layers/{project_id}")
    print(f"   FastAPI Docs: {fastapi_url}/docs")
else:
    print("\n⚠️  Failed to push to FastAPI")
    print("   Layers are still available via direct GEE tile URLs")
    print("   You can still add them to MapStore manually")


Registering layers with FastAPI...
Pushing to: http://fastapi:8000/layers/register
✓ Successfully pushed to FastAPI
  Status: success
  Project ID: sentinel_analysis_20251020_174835
  Layers Count: 5
  Message: Layers registered successfully

📡 Data is now available in FastAPI service
   Access layers: http://fastapi:8000/layers/sentinel_analysis_20251020_174835
   FastAPI Docs: http://fastapi:8000/docs


In [10]:
# Step 9: Generate MapStore Catalog Configuration
# Create configuration that MapStore can use to add these layers to its catalog

mapstore_config = {
    'catalog': {
        'services': {
            'GEE_Sentinel_Analysis': {
                'type': 'tile',
                'title': f'Sentinel-2 Analysis - {project_id}',
                'description': 'Cloudless Sentinel-2 composite and derived indices',
                'url': fastapi_url,
                'layers': []
            }
        }
    },
    'layers': []
}

# Add each layer to MapStore configuration
for layer_name, layer_info in layers_data['layers'].items():
    # Format layer for MapStore
    mapstore_layer = {
        'type': 'tile',
        'name': f'{project_id}_{layer_name}',
        'title': layer_info['name'],
        'description': layer_info['description'],
        'url': layer_info['tile_url'],
        'visibility': False,  # Start hidden, user can toggle
        'opacity': 1.0,
        'format': 'image/png',
        'transparent': True,
        'tileSize': 256,
        'metadata': {
            'analysis_date': datetime.now().isoformat(),
            'source': 'Google Earth Engine',
            'satellite': 'Sentinel-2',
            'date_range': f"{date_start_end[0]} to {date_start_end[1]}"
        }
    }
    
    mapstore_config['layers'].append(mapstore_layer)
    mapstore_config['catalog']['services']['GEE_Sentinel_Analysis']['layers'].append(layer_name)

print("✓ MapStore catalog configuration generated")
print(f"  Service name: GEE_Sentinel_Analysis")
print(f"  Total layers for catalog: {len(mapstore_config['layers'])}")

# Display the configuration
print("\n📋 MapStore Configuration:")
print(json.dumps(mapstore_config, indent=2))


✓ MapStore catalog configuration generated
  Service name: GEE_Sentinel_Analysis
  Total layers for catalog: 5

📋 MapStore Configuration:
{
  "catalog": {
    "services": {
      "GEE_Sentinel_Analysis": {
        "type": "tile",
        "title": "Sentinel-2 Analysis - sentinel_analysis_20251020_174835",
        "description": "Cloudless Sentinel-2 composite and derived indices",
        "url": "http://fastapi:8000",
        "layers": [
          "true_color",
          "false_color",
          "ndvi",
          "evi",
          "ndwi"
        ]
      }
    }
  },
  "layers": [
    {
      "type": "tile",
      "name": "sentinel_analysis_20251020_174835_true_color",
      "title": "True Color",
      "description": "TRUE_COLOR visualization from Sentinel-2",
      "url": "https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/6951b6c1d53c24dc168a5d32691b2608-325a0c6a0822cbed1b71fcc38ba44a8b/tiles/{z}/{x}/{y}",
      "visibility": false,
      "opacity": 1.0,
      "form

In [11]:
# Step 10: Test Direct Access to Layers
# Verify that the layers are accessible through FastAPI

def test_layer_access(project_id, layer_name):
    """Test if layer is accessible through FastAPI"""
    try:
        # Get layer information
        response = requests.get(
            f"{fastapi_url}/layers/{project_id}",
            timeout=10
        )
        
        if response.status_code == 200:
            result = response.json()
            print(f"✓ Layer accessible: {layer_name}")
            print(f"  Available layers: {list(result.get('layers', {}).keys())}")
            return True
        else:
            print(f"✗ Layer not accessible: {response.status_code}")
            return False
    except Exception as e:
        print(f"✗ Error testing access: {e}")
        return False

# Test access
print("Testing layer access through FastAPI...")
test_layer_access(project_id, 'ndvi')


Testing layer access through FastAPI...
✓ Layer accessible: ndvi
  Available layers: ['true_color', 'false_color', 'ndvi', 'evi', 'ndwi']


True

localhost:8888

In [12]:
# Step 11: Add Layers to MapStore Configuration
# Automatically add the GEE layers to MapStore's localConfig.json

import sys
import os
import json
import requests

# Add the current directory to path to import our script
sys.path.append('/usr/src/app/notebooks')

def add_gee_layers_to_mapstore_config(fastapi_url="http://fastapi:8000"):
    """Add GEE layers to MapStore configuration"""
    try:
        # Correct path to MapStore config (host filesystem)
        mapstore_config_path = '/Users/miqbalf/gis-carbon-ai/mapstore/localConfig.json'
        
        print("🔗 Adding GEE layers to MapStore configuration...")
        print(f"📖 Loading MapStore configuration from: {mapstore_config_path}")
        
        # Load current MapStore configuration
        if not os.path.exists(mapstore_config_path):
            print(f"❌ MapStore config not found: {mapstore_config_path}")
            print("   Trying alternative path...")
            # Try alternative path
            mapstore_config_path = '/Users/miqbalf/gis-carbon-ai/mapstore/localConfig.json'
            if not os.path.exists(mapstore_config_path):
                print(f"❌ MapStore config not found: {mapstore_config_path}")
                return False
        
        with open(mapstore_config_path, 'r') as f:
            config = json.load(f)
        print("  ✅ Configuration loaded")
        
        # Create backup
        backup_path = mapstore_config_path.replace('.json', '.backup.json')
        with open(backup_path, 'w') as f:
            json.dump(config, f, indent=2)
        print(f"  💾 Backup created: {backup_path}")
        
        # Get registered projects from FastAPI
        print("🔍 Fetching GEE layers from FastAPI...")
        projects = []
        
        # Try to get the current project
        try:
            response = requests.get(f"{fastapi_url}/layers/{project_id}", timeout=5)
            if response.status_code == 200:
                project_data = response.json()
                if project_data.get('status') == 'success' and project_data.get('layers'):
                    projects.append({
                        'project_id': project_id,
                        **project_data
                    })
                    print(f"  ✅ Found current project: {project_id} ({len(project_data['layers'])} layers)")
        except Exception as e:
            print(f"  ⚠️  Could not fetch current project: {e}")
        
        if not projects:
            print("  ⚠️  No GEE projects found. Using current analysis data...")
            # Use current analysis data
            projects = [{
                'project_id': project_id,
                'project_name': layers_data.get('project_name', 'GEE Analysis'),
                'layers': layers_data['layers']
            }]
        
        # Ensure catalogServices exists
        if 'catalogServices' not in config:
            config['catalogServices'] = {'services': []}
        
        if 'services' not in config['catalogServices']:
            config['catalogServices']['services'] = []
        
        # Add GEE tile service
        gee_service = {
            "type": "tile",
            "title": "GEE Analysis Layers",
            "description": "Google Earth Engine analysis layers from FastAPI service",
            "url": f"{fastapi_url}/tiles/gee/{{layer_name}}/{{z}}/{{x}}/{{y}}",
            "format": "image/png",
            "transparent": True,
            "tileSize": 256,
            "authRequired": False
        }
        
        # Check if GEE service already exists
        existing_gee_service = None
        for i, service in enumerate(config['catalogServices']['services']):
            if service.get('title') == 'GEE Analysis Layers':
                existing_gee_service = i
                break
        
        if existing_gee_service is not None:
            print("  🔄 Updating existing GEE service...")
            config['catalogServices']['services'][existing_gee_service] = gee_service
        else:
            print("  ➕ Adding new GEE service...")
            config['catalogServices']['services'].append(gee_service)
        
        # Ensure map.layers exists
        if 'map' not in config:
            config['map'] = {}
        if 'layers' not in config['map']:
            config['map']['layers'] = []
        
        # Remove existing GEE layers
        config['map']['layers'] = [
            layer for layer in config['map']['layers'] 
            if not (layer.get('name', '').startswith('sentinel_analysis_') or 
                   layer.get('name', '').startswith('test_project_'))
        ]
        
        # Add new GEE layers
        layer_count = 0
        for project in projects:
            print(f"  📁 Processing project: {project['project_id']}")
            
            for layer_name, layer_data in project['layers'].items():
                mapstore_layer = {
                    "type": "tile",
                    "name": f"{project['project_id']}_{layer_name}",
                    "title": layer_data.get('name', layer_name.upper()),
                    "description": layer_data.get('description', f'{layer_name} from GEE analysis'),
                    "url": layer_data.get('tile_url', f"{fastapi_url}/tiles/gee/{layer_name}/{{z}}/{{x}}/{{y}}"),
                    "format": "image/png",
                    "transparent": True,
                    "tileSize": 256,
                    "visibility": False,
                    "opacity": 1.0,
                    "metadata": {
                        "source": "Google Earth Engine",
                        "project_id": project['project_id'],
                        "layer_name": layer_name,
                        "analysis_date": datetime.now().isoformat(),
                        "satellite": "Sentinel-2"
                    }
                }
                config['map']['layers'].append(mapstore_layer)
                layer_count += 1
                print(f"    ➕ Added layer: {mapstore_layer['name']}")
        
        print(f"  ✅ Added {layer_count} GEE layers to MapStore configuration")
        
        # Save updated configuration
        print("💾 Saving updated configuration...")
        with open(mapstore_config_path, 'w') as f:
            json.dump(config, f, indent=2)
        print(f"  ✅ Updated config saved: {mapstore_config_path}")
        
        print("\n📊 Summary:")
        print(f"  Projects processed: {len(projects)}")
        print(f"  Total layers added: {layer_count}")
        print(f"  MapStore layers: {len(config['map']['layers'])}")
        print(f"  Catalog services: {len(config['catalogServices']['services'])}")
        
        return True
        
    except Exception as error:
        print(f"\n❌ Error: {error}")
        return False

# Add layers to MapStore
success = add_gee_layers_to_mapstore_config(fastapi_url="http://fastapi:8000")

if success:
    print("\n✅ Successfully added GEE layers to MapStore!")
    print("\n📋 Next steps:")
    print("  1. Restart MapStore container:")
    print("     docker-compose -f docker-compose.dev.yml restart mapstore")
    print("  2. Open MapStore: http://localhost:8082/mapstore")
    print("  3. Go to Catalog → Search for 'GEE' or your project name")
    print("  4. Add layers to your map")
    print("\n🎯 Your layers will appear as:")
    for layer_name in layers_data['layers'].keys():
        print(f"  - {project_id}_{layer_name}")
else:
    print("\n⚠️  Failed to add layers to MapStore configuration")
    print("   You can still add them manually using the tile URLs above")


🔗 Adding GEE layers to MapStore configuration...
📖 Loading MapStore configuration from: /Users/miqbalf/gis-carbon-ai/mapstore/localConfig.json
❌ MapStore config not found: /Users/miqbalf/gis-carbon-ai/mapstore/localConfig.json
   Trying alternative path...
❌ MapStore config not found: /Users/miqbalf/gis-carbon-ai/mapstore/localConfig.json

⚠️  Failed to add layers to MapStore configuration
   You can still add them manually using the tile URLs above


In [13]:
# Step 12: Restart MapStore to Apply Changes
# Restart MapStore container to load the new configuration

import subprocess
import time

def restart_mapstore():
    """Restart MapStore container to apply configuration changes"""
    try:
        print("🔄 Restarting MapStore container...")
        
        # Restart MapStore container
        result = subprocess.run([
            'docker-compose', '-f', 'docker-compose.dev.yml', 'restart', 'mapstore'
        ], capture_output=True, text=True, cwd='/Users/miqbalf/gis-carbon-ai')
        
        if result.returncode == 0:
            print("  ✅ MapStore container restarted successfully")
            print("  ⏳ Waiting for MapStore to start up...")
            time.sleep(10)  # Wait for MapStore to start
            
            # Test if MapStore is accessible
            try:
                response = requests.get('http://localhost:8082/mapstore', timeout=10)
                if response.status_code == 200:
                    print("  ✅ MapStore is accessible")
                    return True
                else:
                    print(f"  ⚠️  MapStore returned status code: {response.status_code}")
                    return False
            except Exception as e:
                print(f"  ⚠️  MapStore not yet accessible: {e}")
                print("  💡 Wait a few more seconds and try: http://localhost:8082/mapstore")
                return False
        else:
            print(f"  ❌ Failed to restart MapStore: {result.stderr}")
            return False
            
    except Exception as e:
        print(f"  ❌ Error restarting MapStore: {e}")
        return False

# Restart MapStore if layers were added successfully
if success:
    restart_success = restart_mapstore()
    
    if restart_success:
        print("\n🎉 MapStore is ready with your GEE layers!")
        print("\n🌐 Access your layers at:")
        print("   http://localhost:8082/mapstore")
        print("\n📋 How to add layers to your map:")
        print("   1. Click the 'Catalog' button (📁) in the toolbar")
        print("   2. Look for 'GEE Analysis Layers' service")
        print("   3. Click on any layer name to add it to your map")
        print("   4. Use layer controls to toggle visibility and adjust opacity")
    else:
        print("\n⚠️  MapStore restart failed, but layers are configured")
        print("   You can manually restart: docker-compose -f docker-compose.dev.yml restart mapstore")
else:
    print("\n⚠️  Skipping MapStore restart (layers not added)")



⚠️  Skipping MapStore restart (layers not added)


In [14]:
# BONUS: Save configuration to file for later use
output_file = f'/usr/src/app/data/{project_id}_config.json'

try:
    # Ensure data directory exists
    os.makedirs('/usr/src/app/data', exist_ok=True)
    
    # Prepare complete configuration
    complete_config = {
        'project_info': layers_data,
        'mapstore_config': mapstore_config,
        'generated_at': datetime.now().isoformat()
    }
    
    # Save to file
    with open(output_file, 'w') as f:
        json.dump(complete_config, f, indent=2)
    
    print(f"✓ Configuration saved to: {output_file}")
    print(f"  File size: {os.path.getsize(output_file)} bytes")
    print(f"\n  You can use this file to:")
    print(f"  1. Import layers into MapStore")
    print(f"  2. Share analysis configuration with team")
    print(f"  3. Reproduce analysis with same parameters")
except Exception as e:
    print(f"✗ Error saving configuration: {e}")


✓ Configuration saved to: /usr/src/app/data/sentinel_analysis_20251020_174835_config.json
  File size: 8206 bytes

  You can use this file to:
  1. Import layers into MapStore
  2. Share analysis configuration with team
  3. Reproduce analysis with same parameters
